In [13]:
import os
import sys
sys.path.append(os.path.abspath("src"))


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from models.module.Inception import Inception
from models.GoogLeNet import GoogLeNet  # GoogLeNet 모델 불러오기



In [21]:
# CUDA 설정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20
batch_size = 128
learning_rate = 0.001
num_classes = 10
print(device)

cuda


In [16]:
# CIFAR- 10 데이터셋 로딩 및 전처리
transforms = transforms.Compose([
    transforms.Resize((224, 224)), #GoogLeNet 입력 크기에 맞춤
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))
])

train_datset = datasets.CIFAR10(root = './data', train=True, transform=transforms, download=True)
train_loader = DataLoader(train_datset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


In [17]:
# 모델 및 손실함수, 옵티마이저 초기화
model = GoogLeNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [18]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # 학습 모드 활성화

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        print(f"Epoch [{epoch + 1}/{num_epochs}] 시작")  # 에포크 시작 알림

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # 옵티마이저 초기화
            optimizer.zero_grad()

            # 모델에 입력을 넣어 결과 얻기
            outputs, aux1, aux2 = model(inputs)
            
            # 손실 계산 (메인 분류기와 두 개의 보조 분류기의 손실을 모두 사용)
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux1, labels)
            loss3 = criterion(aux2, labels)
            loss = loss1 + 0.3 * loss2 + 0.3 * loss3  # 메인과 보조 분류기 손실 가중치

            # 역전파 및 옵티마이저 업데이트
            loss.backward()
            optimizer.step()

            # 손실 및 정확도 기록
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # 배치 진행 상태 출력
            print(f"Batch [{batch_idx + 1}/{len(train_loader)}] - Loss: {loss.item():.4f}")
        
        # 에포크당 평균 손실과 정확도 출력
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
        print("=" * 50)  # 에포크 구분선

    print("학습 완료")


In [ ]:
# 학습 실행
train(model, train_loader, criterion, optimizer, num_epochs=10)
